## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

/opt/homebrew/Caskroom/miniforge/base/envs/llm-zoomcamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
vector = embedding_model.encode(user_question)

In [6]:
vector[0]

0.07822262

In [7]:
import requests 

In [12]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = [doc for doc in docs_response.json() if doc['course'] == 'machine-learning-zoomcamp']

In [14]:
len(documents)

375

## Q2. Creating the embeddings

In [19]:
import numpy as np
from tqdm import tqdm

In [22]:
embeddings = [embedding_model.encode('{question} {text}'.format(**doc)) for doc in tqdm(documents)]

100%|██████████| 375/375 [00:41<00:00,  9.04it/s]


In [23]:
X = np.array(embeddings)

In [25]:
X.shape

(375, 768)

In [35]:
np.save('X.npy', X) # Save the array for later use

## Q3. Creating the embeddings

In [28]:
scores = X.dot(vector)

In [31]:
scores.max()

0.6506574

### Vector search

In [33]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c